In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import target_encoding as te
import gc

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 
import time

#Images
import cv2
from PIL import Image
import zipfile
import concurrent.futures

In [2]:
train_df = pd.read_csv("/mnt/0d28870f-7d1c-4a10-841a-0195e11cdce3/Jonas/Avito/data/train.csv", parse_dates=["activation_date"])#, nrows=1000)
test_df = pd.read_csv("/mnt/0d28870f-7d1c-4a10-841a-0195e11cdce3/Jonas/Avito/data/test.csv", parse_dates=["activation_date"])#, nrows=1000)
trainindex = train_df.index
testindex = test_df.index
test_id = test_df["item_id"].values
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)


Train file rows and columns are :  (1503424, 18)
Test file rows and columns are :  (508438, 17)


In [3]:
train_y = train_df.deal_probability.copy()
train_df.drop("deal_probability",axis=1, inplace=True)
# Target encode the categorical variables #
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3", "image_top_1"]
for col in cat_vars:
    train_df[col], test_df[col] = te.target_encode(train_df[col], test_df[col], train_y, min_samples_leaf=100, smoothing=10, noise_level=0.01)

In [4]:
# Image Featues
cols_to_add = ['width', 'height', 'img_mean_color', 'img_std_color', 'avg_red', 'avg_green', 'avg_blue', 'blurryness']
train_df = pd.concat([train_df,pd.DataFrame(columns=cols_to_add)])
test_df = pd.concat([test_df,pd.DataFrame(columns=cols_to_add)])
train_df[cols_to_add] = train_df[cols_to_add].apply(pd.to_numeric)
test_df[cols_to_add] = test_df[cols_to_add].apply(pd.to_numeric)
train_df.fillna(0,inplace=True)
test_df.fillna(0,inplace=True)

In [5]:
train_df.head()

,activation_date,avg_blue,avg_green,avg_red,blurryness,category_name,city,description,height,image,...,param_1,param_2,param_3,parent_category_name,price,region,title,user_id,user_type,width
0,2017-03-28,0.0,0.0,0.0,0.0,0.200410,0.121560,"Кокон для сна малыша,пользовались меньше месяц...",0.0,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,...,0.091038,0.140372,0.140869,0.076021,400.0,0.121401,Кокоби(кокон для сна),e00f8ff2eaf9,0.149016,0.0
1,2017-03-26,0.0,0.0,0.0,0.0,0.191746,0.141313,"Стойка для одежды, под вешалки. С бутика.",0.0,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,...,0.123041,0.140194,0.138528,0.177335,3000.0,0.138003,Стойка для Одежды,39aeb48f0017,0.151873,0.0
2,2017-03-20,0.0,0.0,0.0,0.0,0.171280,0.126960,"В хорошем состоянии, домашний кинотеатр с blu ...",0.0,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,...,0.122309,0.136329,0.139835,0.176872,4000.0,0.136797,Philips bluray,91e2f88dd6e3,0.149318,0.0
3,2017-03-25,0.0,0.0,0.0,0.0,0.197608,0.133087,Продам кресло от0-25кг,0.0,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,...,0.334378,0.140295,0.141623,0.075937,2200.0,0.142101,Автокресло,bf5cccea572d,0.123816,0.0
4,2017-03-16,0.0,0.0,0.0,0.0,0.278866,0.137029,Все вопросы по телефону.,0.0,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,...,0.283025,0.366105,0.375922,0.263922,40000.0,0.146277,"ВАЗ 2110, 2003",ef50846afc0b,0.146630,0.0


In [6]:
def get_image_features(df, image_id, file):
    image = Image.open(file)
    dat = np.array(image)
    #imshow(image)
    df.loc[df['image'] == image_id, 'width'] = np.size(dat, 0)
    df.loc[df['image'] == image_id, 'height'] =  np.size(dat, 1)
    df.loc[df['image'] == image_id, 'img_mean_color'] = np.mean(dat[1].flatten())
    df.loc[df['image'] == image_id, 'img_std_color'] = np.std(dat[1].flatten())
    average_color = [dat[:, :, i].mean() for i in range(dat.shape[-1])]
    df.loc[df['image'] == image_id, 'avg_red'] = average_color[0]/255
    df.loc[df['image'] == image_id, 'avg_green'] = average_color[1]/255
    df.loc[df['image'] == image_id, 'avg_blue'] = average_color[2]/255
    image = cv2.cvtColor(dat, cv2.COLOR_BGR2GRAY)
    df.loc[df['image'] == image_id, 'blurryness'] = cv2.Laplacian(image, cv2.CV_64F).var()
    return    

In [7]:
train_zip = zipfile.ZipFile('/mnt/0d28870f-7d1c-4a10-841a-0195e11cdce3/Jonas/Avito/data/train_jpg.zip', 'r')
test_zip = zipfile.ZipFile('/mnt/0d28870f-7d1c-4a10-841a-0195e11cdce3/Jonas/Avito/data/test_jpg.zip', 'r')

def insert_image_features(zipfile):
    df = test_df
    if(zipfile == train_zip):
        df = train_df
    files_in_zip = zipfile.namelist()
    for i,filename in enumerate(files_in_zip):
        if filename.endswith('.jpg'):  
            if (i%1000 == 0): print("Image Number:", i)
            image_id = filename.split('/')[-1].split('.')[0]
            if (image_id in df.image.values):
                file = zipfile.open(files_in_zip[i])
                get_image_features(df, image_id, file)
    return
        
insert_image_features(train_zip)
insert_image_features(test_zip)

Image Number: 1000
Image Number: 2000
Image Number: 3000
Image Number: 4000
Image Number: 5000
Image Number: 6000
Image Number: 7000
Image Number: 8000
Image Number: 9000
Image Number: 10000
Image Number: 11000
Image Number: 12000
Image Number: 13000
Image Number: 14000


KeyboardInterrupt: 

In [ ]:
train_df.head()

In [ ]:
# Combine train and test for rest of preprocessing

df = pd.concat([train_df,test_df],axis=0)
del train_df, test_df
gc.collect()

df.head()

In [ ]:
# Simple Feature Engineering

# Time Data
df["activation_weekday"] = df["activation_date"].dt.weekday
df["activation_monthday"] = df["activation_date"].dt.day

# Price
## Replace Nan with mean in price
#categories = df.category_name.unique()
#region = df.region.unique()
#param1 = df.param_1.unique()
#
#
#df["price_new"] = df["price"].values
#
#for cat in categories:
#    for reg in region:
#        cur_df = df.loc[(df["category_name"] == cat)  & (df["region"] == reg)]["price_new"]
#        cur_df.fillna(np.nanmean(cur_df.values), inplace=True)
#
#
#df["price"] = pd.isna(df["price"])
df["price"] = np.log(df["price"]+0.001)
df["price"].fillna(-999,inplace=True)
df["image_top_1"].fillna(-999,inplace=True)

#Drop Cols
cols_to_drop = ["item_id", "user_id", "activation_date", "image"]
df.drop(cols_to_drop, axis=1,inplace=True)

In [ ]:
# Text Features
df['text_feat'] = df.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features


# Meta Text Features
textfeats = ["description","text_feat", "title"]
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('nicapotato') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words

In [ ]:
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
russian_stop = set(stopwords.words('russian'))

tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}
def get_col(col_name): return lambda x: x[col_name]
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            #max_features=7000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])
    
start_vect=time.time()
vectorizer.fit(df.loc[trainindex,:].to_dict('records'))
ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

# Drop Text Cols
df.drop(textfeats, axis=1,inplace=True)

In [ ]:
train_X = hstack([csr_matrix(df.head(trainindex.shape[0]).values),ready_df[0:trainindex.shape[0]]]) # Sparse Matrix
test_X = hstack([csr_matrix(df.tail(testindex.shape[0]).values),ready_df[trainindex.shape[0]:]])
tfvocab = df.columns.tolist() + tfvocab
for shape in [train_X,test_X]:
    print("{} Rows and {} Cols".format(*shape.shape))
print("Feature Names Length: ",len(tfvocab))
del df
gc.collect();

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y, feature_name=tfvocab)
    lgval = lgb.Dataset(val_X, label=val_y, feature_name=tfvocab)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=20, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:

# Splitting the data for model training#
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)


# Training the model #
pred_test, model, evals_result = run_lgb(X_train, y_train, X_val, y_val, test_X)

# Making a submission file #
pred_test[pred_test>1] = 1
pred_test[pred_test<0] = 0
sub_df = pd.DataFrame({"item_id":test_id})
sub_df["deal_probability"] = pred_test
sub_df.to_csv("baseline_lgb.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()